In [4]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.models import load_model

# --- CONFIGURATION ---
# Define the new 3-class labels
CLASS_LABELS = ["Pothole", "Manhole", "Normal"]
IMG_SHAPE = 299

# File paths for your two saved models
MODEL_V3_PATH = "InceptionV3_3_class_pothole_manhole_E40_LR-5.keras"
MODEL_IRV2_PATH = "InceptionResNetV2_3_class_pothole_manhole_E20_LR-5.keras"

# --- UTILITY FUNCTIONS ---

def load_and_preprocess_image(image_path):
    """Loads, resizes, and normalizes a single image for model input."""
    img_raw = tf.io.read_file(image_path)
    # Assuming the test images are still in JPEG format
    img = tf.image.decode_jpeg(img_raw, channels=3)
    img = tf.image.resize(img, (IMG_SHAPE, IMG_SHAPE))
    img = img / 255.0
    # Add a batch dimension (1, 299, 299, 3)
    return np.expand_dims(img, axis=0)

def ensemble_predict(image_input, model_v3, model_irv2):
    """Performs Soft Voting (Averaging Probabilities) ensemble prediction."""
    
    # 1. Get predictions from both models
    prob_v3 = model_v3.predict(image_input, verbose=0)
    prob_irv2 = model_irv2.predict(image_input, verbose=0)
    
    # 2. Average the probabilities
    avg_probabilities = (prob_v3 + prob_irv2) / 2
    
    # 3. Determine the final class
    predicted_index = np.argmax(avg_probabilities, axis=1)[0]
    confidence = avg_probabilities[0][predicted_index] * 100
    
    return predicted_index, confidence, avg_probabilities[0]

# --- EXECUTION ---

# Load both models once
try:
    model_v3 = load_model(MODEL_V3_PATH)
    model_irv2 = load_model(MODEL_IRV2_PATH)
    print("Ensemble models loaded successfully.")
except Exception as e:
    print(f"ERROR: Could not load one or both models. Check filenames. {e}")
    exit()

# Define a path to a NEW image for a real-world test
# Replace this with the path to an image you have NOT trained on.
test_image_path = r"C:\Users\ABHILASH\Downloads\imgtewg.jpeg"

# Preprocess the image
input_image = load_and_preprocess_image(test_image_path)

# Get the ensemble prediction
predicted_index, confidence, all_probs = ensemble_predict(input_image, model_v3, model_irv2)

# Output the results
print(f"\n--- ENSEMBLE CLASSIFICATION RESULT ---")
print(f"Input Image: {os.path.basename(test_image_path)}")
print(f"Prediction: {CLASS_LABELS[predicted_index]}")
print(f"Confidence: {confidence:.2f}%")

print("\nDetailed Probabilities:")
for i, label in enumerate(CLASS_LABELS):
    print(f" - {label}: {all_probs[i] * 100:.2f}%")

Ensemble models loaded successfully.

--- ENSEMBLE CLASSIFICATION RESULT ---
Input Image: imgtewg.jpeg
Prediction: Normal
Confidence: 99.94%

Detailed Probabilities:
 - Pothole: 0.05%
 - Manhole: 0.01%
 - Normal: 99.94%
